## IBM Data Science Professional Certificate Capstone Project
### Mexican Restaurant in San Francisco

Włodzimierz Kuczyński

April 2021

## 1. Introduction

The goal of this project will be to find the best location for a Mexican restaurant in San Francisco, CA. As defined by the Investor this will be a premium restaurant, aimed mainly at 30-40 year old clientele. One of the main attractions will be the high quality of the food. At first we assume the the main customers will be the inhabitants of the neighboring areas, with a strong emphasis on clients with Mexican origin. 

Firstly I will analyse the the demographics of San Francisco to identify the market area with the optimum clientele base inline with the above assumptions. 
Secondly I will look into the selected area, examine the competition and pinpoint a more precise location.
The analysis will be valueable for anyone planning to open a similar Mexican restaurant in San Francisco.


## 2. Data

I will use census tracts data obtained from openICPSR for the demographics analysis of the San Francisco - (National Neighborhood Data Archive (NaNDA): Socioeconomic Status and Demographic Characteristics of Census Tracts, United States, 2008-2017). ICPSR is an Inter-university Consortium for Political and Social Research.
Machine learning clustering on the relevant census tracts data will be carried out to identify the preferable neighboorhood.
The key data used to cluster the tracts will be:
* the average number of the population with income above 100k$ per square mile (the highest threshold in the census data),  
* the average number of the Hispanic population per square mile- the majority of Hispanic population in SanFrancisco are of Mexican origin,
* the average number of the 30-40 year old population per square mile,

Additionally the land area of each tract will be considered.

The geojson census tracts file used for the maps was obtained from Metropolitan Transportation Commission (MTC) website, the transportation planning, coordinating and financing agency for the nine-county San Francisco Bay Area.

Information about the the competition in the selected area will be obtained with the Foursquare API. Two key factors will analysed using this data:
* location of other Mexican restaurants - the restaurant should be located as far as possible from the direct competitors,
* location of other restaurants - locating the restaurant next to other restaurants which are not direct competitors could be even beneficial (poossiblity of taking over their clients).

#### Reading information about the demographics of San Francisco census tracts in terms of:
#### 1. Persons per square mile, ACS 2013-2017 (popden13_17)
#### 3. Proportion of people of Hispanic origin, ACS 2013-2017 (phispanic13_17)
#### 3. Proportion of families with income greater than 100K, ACS 2013-2017 (inc_above_100k)
#### 4. Proportion of population 30-49 years of age, ACS 2013-2017 (30-39_pop)
#### 5. Land area of each tract (land_area)



In [4]:
pd.set_option('display.max_rows', 10)
demogr=pd.read_csv('nanda_ses_tract_2008-2017_04P.csv',dtype={'tract_fips10':str})
demogr['trint']=demogr['tract_fips10'].astype(int)
demogr_sf=demogr[(demogr['trint']<=6075990100) & (demogr['trint']>=6075010100)]
demogr.drop(columns=['trint'],inplace=True)
demogr_sf.reset_index(drop=True,inplace=True)
demogr_sf=demogr_sf.fillna(0)
demogr_sf.columns
demogr_sf_res=demogr_sf[['tract_fips10','popden13_17','phispanic13_17','pin5b_13_17','p30_3913_17','p40_4913_17','aland10']]
demogr_sf_res['30-49_pop']=demogr_sf_res['p30_3913_17']+demogr_sf_res['p40_4913_17']
demogr_sf_res.drop(columns=['p30_3913_17','p40_4913_17'],inplace=True)
demogr_sf_res.rename(columns={'pin5b_13_17':'inc_above_100k'},inplace=True)
demogr_sf_res.rename(columns={'tract_fips10':'trctid'},inplace=True)
demogr_sf_res.rename(columns={'aland10':'land_area'},inplace=True)
demogr_sf_res=demogr_sf_res[['trctid','popden13_17','phispanic13_17','inc_above_100k','30-49_pop','land_area']]
demogr_sf_res

<ipython-input-4-e78bb209f127>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demogr_sf_res['30-49_pop']=demogr_sf_res['p30_3913_17']+demogr_sf_res['p40_4913_17']
/home/vladdie/miniconda3/envs/cap/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/home/vladdie/miniconda3/envs/cap/lib/python3.9/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pa

,trctid,popden13_17,phispanic13_17,inc_above_100k,30-49_pop,land_area
0,06075010100,13232.0900,0.070078,0.430274,0.394505,0.299801
1,06075010200,21917.6300,0.030713,0.705506,0.366262,0.199064
2,06075010300,42257.9700,0.092529,0.510823,0.387252,0.103578
3,06075010400,35832.9800,0.117330,0.696329,0.435307,0.129629
4,06075010500,10194.5400,0.106652,0.635531,0.316983,0.263965
...,...,...,...,...,...,...
192,06075980401,0.0000,0.000000,0.000000,0.000000,0.161902
193,06075980501,1277.7750,0.187980,0.312500,0.148338,0.612001
194,06075980600,623.1144,0.173489,0.406250,0.249513,0.823284
195,06075980900,174.7661,0.239669,0.625000,0.632231,1.384708


#### Calculating the the average number of persons of Hispanic origin, with income above 100k USD and aged 30-49 per square mile for each census tract.

In [7]:
demogr_sf_res_avg=pd.DataFrame()
demogr_sf_res_avg['trctid']=demogr_sf_res['trctid']
demogr_sf_res_avg['avg_hisp']=demogr_sf_res['popden13_17']*demogr_sf_res['phispanic13_17']
demogr_sf_res_avg['avg_inc_above_100k']=demogr_sf_res['popden13_17']*demogr_sf_res['inc_above_100k']
demogr_sf_res_avg['avg_30-49_pop']=demogr_sf_res['popden13_17']*demogr_sf_res['30-49_pop']
demogr_sf_res_avg['land_area']=demogr_sf_res['land_area']
demogr_sf_res_avg

,trctid,avg_hisp,avg_inc_above_100k,avg_30-49_pop,land_area
0,06075010100,927.280282,5693.425998,5220.121250,0.299801
1,06075010200,673.152074,15463.014831,8027.589366,0.199064
2,06075010300,3910.093326,21586.323342,16364.464002,0.103578
3,06075010400,4204.300087,24951.551587,15598.339407,0.129629
4,06075010500,1087.266095,6478.947501,3231.491081,0.263965
...,...,...,...,...,...
192,06075980401,0.000000,0.000000,0.000000,0.161902
193,06075980501,240.195549,399.304688,189.542075,0.612001
194,06075980600,108.103664,253.140225,155.474939,0.823284
195,06075980900,41.886091,109.228813,110.492619,1.384708
